In [42]:
# This script calculates average lick count and average frequency across all trials in a session.

import os
import numpy as np
from scipy.io import loadmat

# Set the working directory to the folder containing the .mat files
os.chdir('/Users/jun/Documents/Work/Project/Intralingual parasympathetic ganglion neurons/Behavior/ILPG_silencing_#2/SessionData_LickToGetReward/')

# Get a list of all .mat files in the current directory that end with "adjusted_data.mat"
mat_files = [f for f in os.listdir() if f.endswith('adjusted_data.mat')]

for mat_file in mat_files:
    # Load the original data
    mat_contents = loadmat(mat_file)
    SessionData = mat_contents['adjusted_SessionData']

    n_trials = SessionData['nTrials'][0, 0][0, 0]
    raw_events = SessionData['RawEvents'][0, 0]['Trial'][0, 0]

    lickometer = {
        'BNCHigh': [np.nan] * n_trials,
        'BNCLow': [np.nan] * n_trials
    }

    for i in range(n_trials):
        trial = raw_events[0, i]['Events'][0, 0]

        if 'BNC1High' not in trial.dtype.names or 'BNC1Low' not in trial.dtype.names:
            lickometer['BNCHigh'][i] = np.nan
            lickometer['BNCLow'][i] = np.nan
        else:
            lickometer['BNCHigh'][i] = trial['BNC1High'][0, 0]
            lickometer['BNCLow'][i] = trial['BNC1Low'][0, 0]

    total_drinking_duration = 0.0
    total_BNC1High_count = 0.0

    trials = SessionData["RawEvents"][0, 0]["Trial"]
    num_trials = SessionData['nTrials'][0, 0][0, 0]

    for trial in range(num_trials):
        state_info = None
        if trial < len(trials.item()[0]):
            state_info = trials.item()[0, trial]["States"][0, 0]

        if state_info is not None and "Drinking" in state_info.dtype.fields.keys():
            drinking_start = state_info["Drinking"][0][0][0][0]
            drinking_end = state_info["Drinking"][0][0][0][1]

            if not (np.isnan(drinking_start) or np.isnan(drinking_end)):
                drinking_duration = drinking_end - drinking_start
                total_drinking_duration += drinking_duration
                # print(f"Trial: {trial}, drinking_duration: {drinking_duration}, total_drinking_duration: {total_drinking_duration}")

                if not np.isnan(lickometer["BNCHigh"][trial]).all():
                    BNC1High_count = sum([1 for l in lickometer["BNCHigh"][trial][0] if l >= drinking_start and l <= drinking_end])
                else:
                    BNC1High_count = 0
                total_BNC1High_count += BNC1High_count

    print(f"Total drinking duration: {total_drinking_duration}")

    average_BNC1High_count = total_BNC1High_count / num_trials if num_trials > 0 else np.nan
    average_frequency = (total_BNC1High_count / total_drinking_duration) if total_drinking_duration > 0 else np.nan

    print("File:", mat_file)
    print("Average BNC1High count during drinking periods:", average_BNC1High_count)
    print("Average frequency:", average_frequency)
   


Total drinking duration: 415.0
File: ILPG_silencing_#2_LickToGetReward_Training_20230413_162800_2023_04_24_Cue0_adjusted_data.mat
Average BNC1High count during drinking periods: 6.676470588235294
Average frequency: 1.6409638554216868
Total drinking duration: 280.0
File: ILPG_silencing_#2_LickToGetReward_Training_20230414_153200_2023_04_24_Cue0_adjusted_data.mat
Average BNC1High count during drinking periods: 3.8256880733944953
Average frequency: 1.4892857142857143
Total drinking duration: 375.0
File: ILPG_silencing_#2_LickToGetReward_Training_20230411_165753_2023_04_24_Cue0_adjusted_data.mat
Average BNC1High count during drinking periods: 8.725274725274724
Average frequency: 2.1173333333333333
Total drinking duration: 320.0
File: ILPG_silencing_#2_LickToGetReward_Training_20230421_132825_2023_04_24_Cue0_adjusted_data.mat
Average BNC1High count during drinking periods: 6.433962264150943
Average frequency: 2.13125
Total drinking duration: 260.0
File: ILPG_silencing_#2_LickToGetReward_Tra